# OpenAI Function Calling

**Notes**:
- LLM's don't always produce the same results. The results you see in this notebook may differ from the results you see in the video.
- Notebooks results are temporary. Download the notebooks to your local machine if you wish to save your results.

In [ ]:
import json
import boto3

bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

In [ ]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [ ]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location. Use fahrenheit by default.",
        "input_schema": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [ ]:
modelId = "anthropic.claude-3-5-sonnet-20240620-v1:0"
body = {
    "messages": messages,
    "tools": functions,
    "max_tokens": 1024,
    "temperature": 0,
    "anthropic_version": "bedrock-2023-05-31"
}
response = bedrock_runtime.invoke_model(
    body=json.dumps(body),
    modelId=modelId
)
response_body = json.loads(response.get("body").read())


In [ ]:
print(response)

In [ ]:
print(response_body)

In [ ]:
response_message = response_body["content"][0]["text"]

In [ ]:
response_message

In [ ]:
response_body["content"][1]["input"]

In [ ]:
args = response_body["content"][1]["input"]

In [ ]:
get_current_weather(**args)

In [ ]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [ ]:
body = {
    "messages": messages,
    "tools": functions,
    "max_tokens": 1024,
    "temperature": 0,
    "anthropic_version": "bedrock-2023-05-31"
}
response = bedrock_runtime.invoke_model(
    body=json.dumps(body),
    modelId=modelId
)

In [ ]:
json.loads(response.get("body").read())

In [ ]:
body["tool_choice"] = {"type": "auto"}

In [ ]:
response = bedrock_runtime.invoke_model(
    body=json.dumps(body),
    modelId=modelId
)

In [ ]:
json.loads(response.get("body").read())

In [ ]:
body["tool_choice"] = {"type": "tool", "name": "get_current_weather"}

In [ ]:
response = bedrock_runtime.invoke_model(
    body=json.dumps(body),
    modelId=modelId
)

In [ ]:
json.loads(response.get("body").read())

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]
body = {
    "messages": messages,
    "tools": functions,
    "max_tokens": 1024,
    "temperature": 0,
    "anthropic_version": "bedrock-2023-05-31"
}
response = bedrock_runtime.invoke_model(
    body=json.dumps(body),
    modelId=modelId
)
response_body = json.loads(response.get("body").read())
print(response_body)

In [ ]:
body["messages"].append({
    "role": "assistant",
    "content": response_body.get("content")})

In [ ]:
tool_use_content = response_body["content"][1]
args = tool_use_content["input"]
observation = get_current_weather(**args)
entry = {
    "tool_use_id": tool_use_content["id"],
    "content": [{"type": "text", "text": json.dumps(observation)}],
    "type": "tool_result"}
body["messages"].append({
    "role": "user",
    "content": [entry]
})


In [ ]:
response = bedrock_runtime.invoke_model(
    body=json.dumps(body),
    modelId=modelId
)
response_body = json.loads(response.get("body").read())
print(response_body.get("content")[0]["text"])